In [1]:
!pip install -q opencv-python-headless

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers, models
import keras
import matplotlib.pyplot as plt
import cv2
import os
import random
tf.config.list_physical_devices('GPU')

2024-08-12 15:35:57.712042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 15:35:57.724613: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 15:35:57.728522: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-12 15:35:57.739276: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1723491359.383800  4064

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
#dataset = "ImageNet"
dataset = "Games"

In [7]:
M = 150 #max labels
N={ #max samples per split #This block defines N according to testing needs
    'test':9999999, #only used for evaluation, so more only increases granularity
    'train':1040, 
    'val':65
} 

#N={ #max samples per split #This block defines N according to the maximums of the ImageNet data
#    'test':9999999,
#    'train':1040,
#    'val':65
#} 

In [8]:
class My_Custom_Generator(keras.utils.Sequence):

    def __init__(self, image_filenames, labels, batch_size, img_size) :
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = img_size


    def __len__(self) :
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int32)


    def __getitem__(self, idx) :
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]

        return np.array([
                cv2.resize(cv2.imread(str(file_name)), self.image_size)
                    for file_name in batch_x])/255.0, np.array(batch_y)

In [9]:
if dataset == 'ImageNet':
    target = "./Data/150_subset/"
elif dataset == 'Games':
    target = "./Data/Games/DATA BACKUP/"

def getData(target, n={} ,k=99999999, m=99999999):
    data = {}
    dataLabels = []
    splits = os.listdir(target)
    for split in splits[:k]:
        data[split] = {'X':[], 'y':[]}
        if dataLabels == []: #avoid potentially different read order
            labels = sorted(os.listdir(target+f'{split}/'))
            dataLabels = labels
        for label in dataLabels[:m]:
            files = os.listdir(target+f'{split}/{label}/')
            random.shuffle(files)
            for file in files[:n[split]]:
                data[split]['X'].append(f'{target}{split}/{label}/{file}')
                y_temp = [0]*min(len(dataLabels), m)
                y_temp[dataLabels.index(label)] = 1
                data[split]['y'].append(np.asarray(y_temp))
        data[split]['X'] = np.asarray(data[split]['X'])
        data[split]['y'] = np.asarray(data[split]['y'])
    return data, dataLabels, splits
data, dataLabels, splits = getData(target, n=N,  m=M)

In [10]:
batch_size = 256
image_size = (256,256)

my_training_batch_generator = My_Custom_Generator(data['train']['X'], data['train']['y'], batch_size, image_size)
my_validation_batch_generator = My_Custom_Generator(data['val']['X'], data['val']['y'], batch_size, image_size)
my_testing_batch_generator = My_Custom_Generator(data['test']['X'], data['test']['y'], batch_size, image_size)

In [14]:
def makeModel(inputShape=(256, 256, 3), labelCount=5):

    keras.backend.clear_session()

    inputs = keras.Input(shape=inputShape)

    x = layers.Conv2D(filters=32, kernel_size=3, strides=3, activation="relu")(inputs) #(85,85,32)

    x = layers.Conv2D(filters=64, kernel_size=3, strides=3, activation="relu")(x) #(28,28,64)

    x = layers.Conv2D(filters=128, kernel_size=8, strides=8, activation="relu")(x) #(3,3,128)

    x = layers.Flatten()(x)

    x = layers.Dense(256, activation='relu')(x)


    outputs = layers.Dense(labelCount, activation='sigmoid')(x)

    return keras.Model(inputs=inputs, outputs=outputs)

model = makeModel(inputShape=(256, 256, 3), labelCount=data['train']['y'][0].shape[0])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 85, 85, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 128)      │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 150)            │        38,550 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 877,526 (3.35 MB)

 Trainable params: 877,526 (3.35 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=5000,
    decay_rate=0.90)

In [16]:
model.compile(optimizer='Adam',
    loss="categorical_crossentropy",
    metrics=['accuracy'])

In [11]:
callbacks = [
keras.callbacks.ModelCheckpoint(
    filepath="callback.keras",
    save_best_only=True,
    monitor="val_accuracy")
]

In [18]:
history = model.fit(my_training_batch_generator,
    epochs = 25,
    verbose = 1,
    callbacks=callbacks,
    validation_data = my_validation_batch_generator,
   )

Epoch 1/25


/home/king/miniconda3/envs/TensorFlows/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1723482253.992232  402250 service.cc:146] XLA service 0x7f043c0035b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723482253.992280  402250 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-08-12 13:04:14.020138: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-12 13:04:14.218255: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 890

  2/609 ━━━━━━━━━━━━━━━━━━━━ 42s 69ms/step - accuracy: 0.0000e+00 - loss: 5.0122   

I0000 00:00:1723482262.377053  402250 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


240/609 ━━━━━━━━━━━━━━━━━━━━ 1:46 289ms/step - accuracy: 0.0019 - loss: 5.0194

2024-08-12 13:05:33.137378: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_530', 8 bytes spill stores, 8 bytes spill loads

2024-08-12 13:05:33.137472: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_359', 116 bytes spill stores, 116 bytes spill loads

2024-08-12 13:05:33.643945: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_540', 380 bytes spill stores, 380 bytes spill loads



609/609 ━━━━━━━━━━━━━━━━━━━━ 203s 314ms/step - accuracy: 0.0026 - loss: 5.0258 - val_accuracy: 0.0062 - val_loss: 5.0086
Epoch 2/25
609/609 ━━━━━━━━━━━━━━━━━━━━ 140s 226ms/step - accuracy: 0.0151 - loss: 5.0050 - val_accuracy: 0.0552 - val_loss: 4.6814
Epoch 3/25
609/609 ━━━━━━━━━━━━━━━━━━━━ 168s 272ms/step - accuracy: 0.0830 - loss: 4.5486 - val_accuracy: 0.2629 - val_loss: 3.6594
Epoch 4/25
609/609 ━━━━━━━━━━━━━━━━━━━━ 153s 248ms/step - accuracy: 0.2751 - loss: 3.5888 - val_accuracy: 0.4438 - val_loss: 2.6686
Epoch 5/25
609/609 ━━━━━━━━━━━━━━━━━━━━ 149s 242ms/step - accuracy: 0.4640 - loss: 2.5239 - val_accuracy: 0.5561 - val_loss: 2.1058
Epoch 6/25
609/609 ━━━━━━━━━━━━━━━━━━━━ 137s 222ms/step - accuracy: 0.5641 - loss: 1.9947 - val_accuracy: 0.6047 - val_loss: 1.8101
Epoch 7/25
609/609 ━━━━━━━━━━━━━━━━━━━━ 146s 237ms/step - accuracy: 0.6005 - loss: 1.7620 - val_accuracy: 0.6419 - val_loss: 1.6032
Epoch 8/25
609/609 ━━━━━━━━━━━━━━━━━━━━ 137s 222ms/step - accuracy: 0.6522 - loss: 1.45

In [19]:
model.save(f'models/cnn_{dataset}_{M:03d}_N-{N["train"]:04d}.keras')

In [12]:
model = keras.saving.load_model(f'models/cnn_{dataset}_{M:03d}_N-{N["train"]:04d}.keras')

I0000 00:00:1723491392.269188  406404 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723491392.269316  406404 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723491392.269388  406404 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723491392.483511  406404 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723491392.483607  406404 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-12

In [13]:
eval = model.evaluate(my_testing_batch_generator,
    batch_size=batch_size,
    callbacks=callbacks,
    return_dict=True
)

/home/king/miniconda3/envs/TensorFlows/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1723491409.521610  406519 service.cc:146] XLA service 0x7f4b90003180 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723491409.521686  406519 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-08-12 15:36:49.542801: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-12 15:36:49.710569: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 890

   2/1789 ━━━━━━━━━━━━━━━━━━━━ 1:58 66ms/step - accuracy: 0.4521 - loss: 3.9669 

I0000 00:00:1723491412.771944  406519 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 892/1789 ━━━━━━━━━━━━━━━━━━━━ 5:41 380ms/step - accuracy: 0.6370 - loss: 2.8133

KeyboardInterrupt: 

In [ ]:
eval